In [1]:
import pandas as pd
from groundingdino.util.inference import Model
from typing import List
import os
import supervision as sv
import cv2
import warnings
from tqdm import tqdm
from ultralytics import YOLO, SAM
import torch
import numpy as np
import matplotlib.pyplot as plt
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.build_sam import build_sam2, build_sam2_video_predictor
import rerun as rr
#from track_utils import sample_points_from_masks
#from video_utils import create_video_from_images
import json
import random
from uuid import uuid4
import pandas as pd
import logging

In [2]:
!pip install supervision==0.22.0

In [3]:
sv.__version__

'0.22.0'

In [4]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if "transformers" in logger.name.lower():
        logger.setLevel(logging.ERROR)

In [5]:
#video_folder = [f"{i}" for i in range(1, 14)]
#video_paths = [os.path.join(f'/mnt/data/Datasets/Innsbruk/test/output_video_part_{f}.mp4') for f in video_folder]
#video_paths

In [6]:
class GLAMModel:
    def __init__(self, grounding_dino_config_path, grounding_dino_checkpoint_path, sam_model_cfg, sam_checkpoint_path, prompt=None):
        if prompt is None:
            # prompt = ['pathways', 'trails', 'walkways', 'sidewalks', 'tracks', 'footpaths', 'routes', 'pedestrian paths', 'walking paths', 'lanes']
            prompt = ['pavement', 'fence', 'cyclepath', 'trees', 'grasses', 'sidewalk', 'buildings', 'skies', 'streetlights']
        self.prompt = prompt
        self.grounding_dino_model = Model(model_config_path=grounding_dino_config_path, model_checkpoint_path=grounding_dino_checkpoint_path)
        self.sam = build_sam2(sam_model_cfg, sam_checkpoint_path, device="cuda")
        self.sam_predictor = SAM2ImagePredictor(self.sam)
        self.yolo = YOLO('yolov8x-seg.pt')
        with open('class_descriptions.json', 'r', encoding='utf-8') as file:
            self.class_names = json.load(file)

        self.class_names += [{'id': 80+i, 'color': self.generate_random_color(), 'name': p} for i, p in enumerate(self.prompt)]
        self.class_dict = {item['id']: item['name'] for item in self.class_names}
        # self.dino_classes = 'pathways . trails . walkways'
        self.dino_classes = self.enhance_class_name(self.prompt)
        # self.dino_classes = str.join(' . ', self.prompt) + ' .'
        self.dino_box_threshold = 0.35
        self.dino_text_threshold = 0.25
        self.class_descriptions = [rr.AnnotationInfo(id=cat["id"], color=cat["color"], label=cat["name"]) for cat in self.class_names]
        self.yolo_classes = [0, 1, 2, 3, 5, 7, 9, 11, 30]  # [0, 1, 2, 3, 5, 7, 9, 10, 11, 13, 14, 15, 16, 56, 60, 67]
        self.persist = []
        self.video_outs = dict()
        
    @staticmethod
    def enhance_class_name(class_names: List[str]) -> List[str]:
        return [f"{class_name}" for class_name in class_names]

    @staticmethod
    def generate_random_color():
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)
        return r, g, b
    
    def add_dino_class(self, _phrase):
        _class_id = max(self.class_dict.keys()) + 1
        self.class_names.append({'id': _class_id, 'color': self.generate_random_color(), 'name': _phrase})
        self.class_dict = {item['id']: item['name'] for item in self.class_names}
        self.class_descriptions = [rr.AnnotationInfo(id=cat["id"], color=cat["color"], label=cat["name"]) for cat in self.class_names]
        return _class_id
    
    def dino_id_to_class_name(self, dino_id):
        return self.class_dict[dino_id]
    
    def phrases2classes(self, phrases: List[str]) -> (np.ndarray, bool):
        class_ids = []
        ret = False
        for phrase in phrases:
            if phrase in self.class_dict.values():
                for k, v in self.class_dict.items():
                    if v == phrase:
                        class_ids.append(k)
            else:
                _class_id = self.add_dino_class(phrase)
                class_ids.append(_class_id)
                ret = True
        return np.array(class_ids), ret

In [7]:
GROUNDING_DINO_CONFIG_PATH = os.path.join('../', "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
GROUNDING_DINO_CHECKPOINT_PATH = os.path.join('../', "weights", "groundingdino_swint_ogc.pth")
SAM_CHECKPOINT_PATH = os.path.join("/home/lnt/PycharmProjects/sam/weights/sam2_hiera_large.pt")
SAM_MODEL_CFG = "sam2_hiera_l.yaml"

In [1]:

glam_model = GLAMModel(grounding_dino_config_path=GROUNDING_DINO_CONFIG_PATH, grounding_dino_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH, sam_model_cfg=SAM_MODEL_CFG, sam_checkpoint_path=SAM_CHECKPOINT_PATH)
frame_pos = 0
df = pd.DataFrame([], columns=['frame number', 'detector', 'x1', 'y1', 'x2', 'y2', 'class_id', 'class_name', 'track_id', 'confidence', 'is_indoor'])

HOME = '/home/lnt/PycharmProjects/sam'
SOURCE_VIDEO_PATH = f"{HOME}/data/world_raw.mp4"
output_path = f"{HOME}/data/masks.mp4"

cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)

cap.set(cv2.CAP_PROP_POS_FRAMES, frame_pos)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

is_indoor = True
# is_indoor = False

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
for frame_number in tqdm(range(frame_pos, total_frames)):
    ret, frame = cap.read()
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    if not ret:
        break

    yolo_results = glam_model.yolo.track(frame, verbose=False, conf=0.6, persist=True, retina_masks=True, classes=glam_model.yolo_classes)[0] # , classes=glam_model.yolo_classes
    yolo_mask_frame = yolo_results.plot()
    
    # 使用 YOLO 结果初始化画面
    annotated_frame = yolo_mask_frame.copy()
    
    if is_indoor:
        count_chair = torch.sum((yolo_results.boxes.cls == 56) | (yolo_results.boxes.cls == 60)).item()
        if count_chair < 1:
            is_indoor = False
    else:
        count_chair = torch.sum((yolo_results.boxes.cls == 56) | (yolo_results.boxes.cls == 60)).item()
        if count_chair > 4:
            is_indoor = True

    # dino_results = glam_model.grounding_dino_model.predict_with_classes(
    dino_results, phrases = glam_model.grounding_dino_model.predict_with_caption(
        image=frame,
        caption=str.join(' . ', glam_model.prompt),
        box_threshold=glam_model.dino_box_threshold,
        text_threshold=glam_model.dino_text_threshold
    )
    dino_results.class_id, ret = glam_model.phrases2classes(phrases)
    
    
    glam_model.sam_predictor.set_image(frame)

    
    for _box, _cls_id, _confidence, _phrase in zip(dino_results.xyxy, dino_results.class_id, dino_results.confidence, phrases):
        _masks, _scores, _logits = glam_model.sam_predictor.predict(
            box=_box,
            multimask_output=True
        )
        _index = np.argmax(_scores)
        _mask = _masks[_index]
        _mask = _mask.astype('bool')
        mask_result = sv.Detections(np.array([_box]), np.array([_mask]), np.array([_confidence]), np.array([_cls_id]))

        mask_annotator = sv.MaskAnnotator()
        box_annotator = sv.BoxAnnotator()
        label_annotator = sv.LabelAnnotator(text_position=sv.Position.CENTER)
        annotated_frame = mask_annotator.annotate(scene=annotated_frame, detections=mask_result)
        annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=mask_result)
        annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=mask_result, labels=[_phrase])
    cv2.putText(annotated_frame, f'Frame: {frame_number}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    out.write(annotated_frame)

out.release()
cap.release()



NameError: name 'GLAMModel' is not defined